In [0]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [0]:
import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [0]:
!pip uninstall -y Pillow
!pip install Pillow==5.3.0
import PIL
print(PIL.PILLOW_VERSION)

In [0]:
# Download required data files
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip

!unzip -qq flower_data.zip

In [0]:
# Define path to directory
data_dir = '/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
data_dir = 'flower_data'
batch_size = 30
valid_size = 0.2

data_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


# Load the datasets with ImageFolder
image_datasets = datasets.ImageFolder(data_dir + '/train', transform = data_transforms)

num_train = len(image_datasets)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# Define the dataloaders
train_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size,
    sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, 
    sampler=valid_sampler)

In [0]:
# Label mapping
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [0]:
# Build the network
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 102),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
# Measure the validation loss and accuracy
def validation(model, dataloader, criterion, device):
    loss = 0
    accuracy = 0
    with torch.no_grad():
        for images, labels in iter(dataloader):
            
            images, labels = images.to(device), labels.to(device) # Move input & label tensors to the GPU
            
            output = model.forward(images)
            loss += criterion(output, labels).item()

            ps = torch.exp(output) # Get the class probabilities from log-softmax
            equality = (labels.data == ps.max(dim=1)[1])
            accuracy += equality.type(torch.FloatTensor).mean()
    
    return loss, accuracy

In [0]:
epochs = 20
print_every = 20
steps = 0
running_loss = 0
train_accuracy = 0

for e in range(epochs):

    model.train()

    for images, labels in iter(train_loader):

        images, labels = images.to(device), labels.to(device)

        steps += 1
        optimizer.zero_grad()
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Get the class probabilities from log-softmax
        ps = torch.exp(output) 
        equality = (labels.data == ps.max(dim=1)[1])
        train_accuracy += equality.type(torch.FloatTensor).mean()

        if steps % print_every == 0:

            model.eval() # Make sure network is in eval mode for inference

            with torch.no_grad():
                valid_loss, valid_accuracy = validation(model, valid_loader, criterion, device)

            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Training Accuracy: {:.3f}".format(train_accuracy/print_every),
                  "Validation Loss: {:.3f}.. ".format(valid_loss/len(valid_loader)),
                  "Validation Accuracy: {:.3f}".format(valid_accuracy/len(valid_loader)))

            running_loss = 0
            train_accuracy = 0
            model.train() # Turn training back on

Epoch: 1/20..  Training Loss: 4.585..  Training Accuracy: 0.028 Validation Loss: 4.504..  Validation Accuracy: 0.027
Epoch: 1/20..  Training Loss: 4.513..  Training Accuracy: 0.035 Validation Loss: 4.422..  Validation Accuracy: 0.028
Epoch: 1/20..  Training Loss: 4.353..  Training Accuracy: 0.080 Validation Loss: 4.213..  Validation Accuracy: 0.133
Epoch: 1/20..  Training Loss: 4.105..  Training Accuracy: 0.122 Validation Loss: 3.898..  Validation Accuracy: 0.152
Epoch: 1/20..  Training Loss: 3.841..  Training Accuracy: 0.128 Validation Loss: 3.489..  Validation Accuracy: 0.240
Epoch: 1/20..  Training Loss: 3.438..  Training Accuracy: 0.227 Validation Loss: 3.103..  Validation Accuracy: 0.288
Epoch: 1/20..  Training Loss: 3.150..  Training Accuracy: 0.287 Validation Loss: 2.821..  Validation Accuracy: 0.317
Epoch: 1/20..  Training Loss: 2.977..  Training Accuracy: 0.273 Validation Loss: 2.636..  Validation Accuracy: 0.349
Epoch: 2/20..  Training Loss: 2.714..  Training Accuracy: 0.307 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# Save the checkpoint
model_save_name = 'classifier.pt'
model_save_path = F"/content/gdrive/My Drive/Colab Notebooks/Lab_Challenge/{model_save_name}"
torch.save(model.state_dict(), model_save_path)

In [0]:
# Load checkpoint and rebuild the model
model_save_name = 'classifier.pt'
model_save_path = F"/content/gdrive/My Drive/Colab Notebooks/Lab_Challenge/{model_save_name}"
model.load_state_dict(torch.load(model_save_path))

In [0]:
# Validation on the test set
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in train_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy achieved on test images is: %d%%' % (100 * correct / total))

Accuracy achieved on test images is: 91%
